In [6]:
#神经网络实验2

from torch.utils.data import Dataset, DataLoader
import os
from setting import BATCH_SIZE, SEED, CHAR_NUMBER
import torch
from torch import nn
from PIL import Image
from torchvision import transforms

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print('Your device:{}'.format(device))
torch.cuda.empty_cache()

class ImageDataSet(Dataset):
    def __init__(self, dir_path):
        super(ImageDataSet, self).__init__()
        self.img_path_list = [f"{dir_path}/{filename}" for filename in os.listdir(dir_path)]
        self.trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.Grayscale()  # 每张图片都会被这行代码灰度化
        ])

    def __getitem__(self, idx):
        image = self.trans(Image.open(self.img_path_list[idx]))
        #print(self.img_path_list[idx])
        label = self.img_path_list[idx].split("_")[0].split('/')[-1]
        label = one_hot_encode(label)
        return image, label

    def __len__(self):
        return len(self.img_path_list)

# 用torch.zeros()函数生成一个4行62列，值全是0的张量。接着循环标签中的各个字符，将字符在SEED中对应的索引获取到，然后将张量中对应位置的0，改成1。最后我们要返回一个一维的列表，长度是4*62=248
def one_hot_encode(label):
    """将字符转为独热码"""
    cols = len(SEED)
    rows = CHAR_NUMBER
    result = torch.zeros((rows, cols), dtype=float)
    for i, char in enumerate(label):
        j = SEED.index(char)
        result[i, j] = 1.0
    return result.view(1, -1)[0]


# 将模型预测的值从一维转成4行62列的二维张量，然后调用torch.argmax()函数寻找每一行最大值（也就是1）的索引。知道索引后就可以从SEED中找到对应的字符
def one_hot_decode(pred_result):
    """将独热码转为字符"""
    pred_result = pred_result.view(-1, len(SEED))
    index_list = torch.argmax(pred_result, dim=1)
    text = "".join([SEED[i] for i in index_list])
    return text


def get_loader(path):
    """加载数据"""
    dataset = ImageDataSet(path)
    dataloader = DataLoader(dataset, BATCH_SIZE, shuffle=True)
    return dataloader
class NeuralNetWork(nn.Module):
    """CNN"""
    def __init__(self):
        super(NeuralNetWork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer5 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=30720, out_features=4096),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=CHAR_NUMBER * len(SEED))
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x


def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            print(f"损失值: {loss:>7f}")


def begintrain():
    model = NeuralNetWork().to(device)
    loss_fn = nn.MultiLabelSoftMarginLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    train_dataloader = get_loader(f"./trainset")
    epoch = 30
    for t in range(epoch):
        print(f"训练周期 {t + 1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        print("\n")
    torch.save(model.state_dict(), f"./CHAPTCHAmodel.pth")
    print("训练完成，模型已保存")


def predict(model, file_path):
    trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Grayscale()
    ])
    with torch.no_grad():
        X = trans(Image.open(file_path)).reshape(1, 1, 100, 160)
        pred = model(X)
        text = one_hot_decode(pred)
        return text


def begintest():
    model = NeuralNetWork().to(device)
    model.load_state_dict(torch.load(f"./CHAPTCHAmodel.pth", map_location=torch.device(device)))
    model.eval()

    correct = 0
    test_dir = f"./testset"
    total = len(os.listdir(test_dir))
    for filename in os.listdir(test_dir):
        file_path = f"{test_dir}/{filename}"
        real_captcha = file_path.split("_")[-1].replace(".png", "")
        pred_captcha = predict(model, file_path)
        if pred_captcha == real_captcha:
            correct += 1
            print(f"{file_path}的预测结果为{pred_captcha}，预测正确")
        else:
            print(f"{file_path}的预测结果为{pred_captcha}，预测错误")
    accuracy = f"{correct / total * 100:.2f}%"
    print(accuracy)

if __name__ == '__main__':
    # 训练集和测试集路径
    train_dataloader = get_loader(f"./trainset")
    test_dataloader = get_loader(f"./testset")
    for X, y in train_dataloader:
        print(X.shape)
        print(y.shape)
        break
    begintrain()
    begintest()

Your device:cuda:0
torch.Size([60, 1, 80, 200])
torch.Size([60, 248])
训练周期 1
-------------------------------
损失值: 0.691904


训练周期 2
-------------------------------
损失值: 0.087315


训练周期 3
-------------------------------
损失值: 0.083628


训练周期 4
-------------------------------
损失值: 0.083863


训练周期 5
-------------------------------
损失值: 0.083476


训练周期 6
-------------------------------
损失值: 0.083762


训练周期 7
-------------------------------
损失值: 0.083258


训练周期 8
-------------------------------
损失值: 0.083631


训练周期 9
-------------------------------
损失值: 0.083307


训练周期 10
-------------------------------
损失值: 0.082715


训练周期 11
-------------------------------
损失值: 0.082863


训练周期 12
-------------------------------
损失值: 0.082546


训练周期 13
-------------------------------
损失值: 0.082654


训练周期 14
-------------------------------
损失值: 0.082951


训练周期 15
-------------------------------
损失值: 0.082603


训练周期 16
-------------------------------
损失值: 0.083019


训练周期 17
-------------------------------
损失值

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument weight in method wrapper_CUDA___slow_conv2d_forward)